## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-08-02-29-32 +0000,nyt,What Is the UNFCCC and Why Is the U.S. Pulling...,https://www.nytimes.com/2026/01/07/climate/wha...
1,2026-01-08-02-19-38 +0000,nyt,The ICE Shooting Came During an Operation Focu...,https://www.nytimes.com/2026/01/07/us/ice-minn...
2,2026-01-08-02-19-17 +0000,nyt,One Video of a Fatal ICE Shooting in Minneapol...,https://www.nytimes.com/2026/01/07/us/minneapo...
3,2026-01-08-02-18-11 +0000,nyt,Trump Pulls Out of Global Climate Treaty,https://www.nytimes.com/2026/01/07/climate/tru...
4,2026-01-08-02-16-34 +0000,nyt,Authorities Often Justify Motorist Shootings b...,https://www.nytimes.com/2026/01/07/us/motorist...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2333/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
18,trump,56
102,venezuela,37
116,oil,32
171,new,22
14,minneapolis,20


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
46,2026-01-08-00-40-04 +0000,nyt,U.S. Details Plan for Venezuela’s Oil Sales Af...,https://www.nytimes.com/2026/01/07/business/en...,154
273,2026-01-07-15-31-28 +0000,nyt,Trump Claims Venezuela Will Send U.S. Millions...,https://www.nytimes.com/2026/01/06/business/ve...,152
344,2026-01-07-10-10-24 +0000,bbc,Trump says Venezuela will be 'turning over' up...,https://www.bbc.com/news/articles/c4grxzxjjd8o...,151
40,2026-01-08-01-00-00 +0000,wsj,The U.S. pumps more oil than any other country...,https://www.wsj.com/business/energy-oil/us-ven...,150
85,2026-01-07-23-24-03 +0000,nyt,"During Maduro Raid in Venezuela, a Close Call ...",https://www.nytimes.com/2026/01/07/us/politics...,134


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
46,154,2026-01-08-00-40-04 +0000,nyt,U.S. Details Plan for Venezuela’s Oil Sales Af...,https://www.nytimes.com/2026/01/07/business/en...
15,85,2026-01-08-02-04-21 +0000,nypost,Minneapolis Mayor Jacob Frey flips out on CNN ...,https://nypost.com/2026/01/07/us-news/minneapo...
351,81,2026-01-07-10-00-00 +0000,wsj,The welfare-fraud scandal that led Minnesota G...,https://www.wsj.com/politics/elections/minneso...
26,64,2026-01-08-01-34-00 +0000,wsj,Trump Administration Draws Up New Legal Justif...,https://www.wsj.com/politics/policy/trump-admi...
40,62,2026-01-08-01-00-00 +0000,wsj,The U.S. pumps more oil than any other country...,https://www.wsj.com/business/energy-oil/us-ven...
67,46,2026-01-08-00-01-55 +0000,nypost,Palisades fire suspect will ‘go to war’ over c...,https://nypost.com/2026/01/07/us-news/palisade...
195,44,2026-01-07-19-13-53 +0000,cbc,ICE agent fatally shoots woman during immigrat...,https://www.cbc.ca/news/world/ice-minneapolis-...
257,39,2026-01-07-16-22-02 +0000,bbc,Canada PM Mark Carney to visit China next week...,https://www.bbc.com/news/articles/cevnek9k32no...
87,38,2026-01-07-23-17-19 +0000,nypost,Marco Rubio making moves on Greenland as Trump...,https://nypost.com/2026/01/07/us-news/marco-ru...
75,36,2026-01-07-23-47-00 +0000,wsj,Senate Republican negotiators say they are clo...,https://www.wsj.com/politics/policy/aca-subsid...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
